In [1]:
import requests
import pandas as pd
import re
import time

from selenium import webdriver

In [2]:
sleep_time = 1

In [3]:
def make_url(category, date, page):
    url = "http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=" + str(category) + "#&date=" + date + " 00:00:00&page=" + str(page)
    return url

In [4]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(sleep_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(sleep_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(sleep_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [5]:
def last_page(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)

    time.sleep(sleep_time)
    pages = driver.find_elements_by_css_selector("#paging ._paging")
    total_page = len(pages)
    
    if len(pages) > 9:
        
        pages[9].click()        
        time.sleep(sleep_time)
        pages = driver.find_elements_by_css_selector("#paging ._paging")
        total_page += len(pages) - 1
        
        while len(pages) > 10:
        
            pages[10].click()        
            time.sleep(sleep_time)
            pages = driver.find_elements_by_css_selector("#paging ._paging")
            
            if len(pages) > 10:
                total_page += len(pages) - 1 
            else:
                total_page += len(pages)
            
    driver.close()   
    
    return total_page

In [6]:
def get_content(url):
    
#     driver =  webdriver.PhantomJS()
#     driver.get(url)
#     time.sleep(sleep_time)
#     content = driver.find_element_by_css_selector("#articleBodyContents").text
    
#     comment = 0
#     comment_element = driver.find_elements_by_css_selector("#articleTitleCommentCount .lo_txt")
#     if int(len(comment_element)) > 0:
#         comment = comment_element[0].text
    
#     likeit = 0
#     likeit_element = driver.find_elements_by_css_selector(".u_likeit_module .u_cnt")
#     if int(len(likeit_element)) > 0:
#         likeit = likeit_element[0].text
    
#     driver.close()
    
    comment, likeit, content = 0,0,"-"
    
    return comment, likeit, content

In [7]:
def article_url_list(url):
    
    driver =  webdriver.PhantomJS()
    driver.get(url)
    
    time.sleep(sleep_time)
    articles = driver.find_elements_by_css_selector("#section_body.section_body ul li")
    
    article_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content"])
    
    for article in articles:
        link = article.find_element_by_css_selector("a").get_attribute("href")
        title = article.find_element_by_css_selector("a").get_attribute("title")
        newspaper = article.find_element_by_css_selector("span").text
        newsid = link.split("aid=")[1]
        comment, likeit, content = get_content(link)  
        
        tmp_dict = {
            "newsid": newsid, 
            "newspaper": newspaper, 
            "title": title, 
            "link": link, 
            "comment": comment, 
            "likeit": likeit, 
            "content": content,
        }
        
        article_df.loc[len(article_df)] = tmp_dict
        
    article_df["date"] = url.split("date=")[1].split(" ")[0]
    article_df["category"] = url.split("sid1=")[1].split("#")[0]
    
    driver.close()
    
    return article_df

In [8]:
def date_category_df(category, date):
    
    url = make_url(category, date, 1)
    last_page_number = last_page(url)
    
    result_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content", "date", "category"])

    for page in range(1, last_page_number + 1):
        link = make_url(category, date, page)
        tmp_df = article_url_list(link)        
        result_df = pd.concat([result_df, tmp_df])
        print(last_page_number, page)
        
    return result_df

In [9]:
def main(category, start_date, days):
    
    result_df = pd.DataFrame(columns=["newsid", "newspaper", "title", "link", "comment", "likeit", "content", "date", "category"])
    
    for day in range(1, days+1):
        if day < 10:
            day = "0" + str(day)
        date = start_date + "-" + str(day) 
        print(date, category)
        tmp_df = date_category_df(category, date)
        result_df = pd.concat([result_df, tmp_df])
        
    return result_df

In [11]:
for category in range(101,106):
    result = main(category,"2016-01", 31)
    result_df = result.reset_index(drop=True)
    result_df.to_csv("./" + str(category) + "_2016-01.csv", index=False, encoding="utf-8")

# for category in range(100,106):
#     result = main(category,"2016-02", 28)
#     result_df = result.reset_index(drop=True)
#     result_df.to_csv("./" + str(category) + "_2016-02.csv", index=False, encoding="utf-8")
    
# for category in range(100,106):
#     result = main(category,"2016-03", 31)
#     result_df = result.reset_index(drop=True)
#     result_df.to_csv("./" + str(category) + "_2016-03.csv", index=False, encoding="utf-8")
    
# for category in range(100,106):
#     result = main(category,"2016-04", 30)
#     result_df = result.reset_index(drop=True)
#     result_df.to_csv("./" + str(category) + "_2016-04.csv", index=False, encoding="utf-8")
    
# for category in range(100,106):
#     result = main(category,"2016-05", 31)
#     result_df = result.reset_index(drop=True)
#     result_df.to_csv("./" + str(category) + "_2016-05.csv", index=False, encoding="utf-8")

# for category in range(100,106):
#     result = main(category,"2016-06", 30)
#     result_df = result.reset_index(drop=True)
#     result_df.to_csv("./" + str(category) + "_2016-06.csv", index=False, encoding="utf-8")

2016-01-01 101
5 1
5 2
5 3
5 4
5 5
2016-01-02 101
3 1
3 2
3 3
2016-01-03 101
5 1
5 2
5 3
5 4
5 5
2016-01-04 101
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
17 11
17 12
17 13
17 14
17 15
17 16
17 17
2016-01-05 101
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
16 16
2016-01-06 101
19 1
19 2
19 3
19 4
19 5
19 6
19 7
19 8
19 9
19 10
19 11
19 12
19 13
19 14
19 15
19 16
19 17
19 18
19 19
2016-01-07 101
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
16 16
2016-01-08 101
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
16 16
2016-01-09 101
3 1
3 2
3 3
2016-01-10 101
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
2016-01-11 101
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
14 14
2016-01-12 101
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
14 14
2016-01-13 101
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
